## Transfer Learning

전이학습(Transfer Learning)은 특정 상황에서 모델을 학습하면서 얻은 지식을 다른 상황에 활용하는 학습방법.

1. 데이터 부족 문제를 어느정도는 해결
2. 학습에 걸리는 시간을 단축할 수 있음
3. 시뮬레이션에서 학습한 모델을 현실에 적용할 수 있게 해준다 (ex.자율주행자동차)

In [1]:
#ResNet50 pretraind된 모델 불러오기(imagenet으로 훈련)

import torchvision.models as models
resnet = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\User/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
82.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


In [2]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
for name, module in resnet.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [6]:
print(list(resnet.children()))

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False), BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False), Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=Tr

In [9]:
#Transfer Learning
import torch.nn as nn

class ResNet(nn.Module):
    def __init__(self, num_category):
        super(ResNet,self).__init__()
        self.layer0 = nn.Sequential(*list(resnet.children())[0:-1]) #특성 추출기만 사용하므로
        self.layer1 = nn.Sequential(nn.Linear(2048, 500), nn.BatchNorm1d(500), nn.ReLU(),
                                     nn.Linear(500, num_category))
    
    def forward(self, x):
        out = self.layer0(x)
        out = out.view(batch_size, -1)
        out = self.layer1(out)
        return out

model = ResNet(10)
model.eval()

ResNet(
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 25

In [11]:
#model의 layer0부분은 학습하지 않음
for param in model.layer0.parameters():
    param.requires_grad = False

#model의 layer1부분은 학습함.
for param in model.layer1.parameters():
    param.requires_grad = True

## Style transfomer(feat. Transfer Learning)

전이학습의 대표적인 활용사례임. 

Style Transfomer에서 활용되는 모델은 주로, 이미지넷에 pretrained된 모델을 쓴다. 그 이유는 필터의 범용성에 있다. pretrained 된 모델의 필터에는 처음에는 이미지의 가로선, 대각선, 색 등에 반응하는 필터도 존재하고, layer가 깊어질수록 복잡하고 추상적인 이미지의 패턴에 반응할 수 있는 필터도 존재한다. 이러한 필터는 단순히 이미지넷에서만 제한되는 것이 아니라, '다른 이미지에 적용할 수 있는 필터'들이다.

Style Transfomer의 구조를 쉽게 설명하면, 어떤 이미지의 스타일을 가져와 콘텐츠가 되는 이미지에 적용한 이미지를 만들어 내는 것이다. 예를 들면 반고흐의 그림의 스타일을 유럽의 건물들의 사진(콘텐츠)에 적용한 이미지를 만들어 내는 것이다. 논문에서 정의한 스타일과 콘텐츠의 정의를 살펴보면, 먼저 스타일은 correlation between the diffrent filter responses라고 정의한다. 즉 서로 다른 필터를 통과한 활성화 텐서들 사이의 상관관계라고 해석할 수 있다. 이는 그람 행렬을 통해 벡터사이의 내적으로 표현할 수 있다. 콘텐츠는 feature responses in higher layers of the network라고 정의한다. 즉 깊은 layer의 활성화 맵들을 가르킨다고 해석할 수 있다. Style Transfomer를 전이학습의 대표적인 활용사례라고 말하는 이유는 스타일과 콘텐츠를 가져오는 모델, 역전파를 통해 이미지를 만들기 위해 사용하는 모델 전부 pretrained된 모델을 사용하기 때문이다.

그렇다면 Style Transfomer는 어떤식으로 학습할까? 먼저 손실함수를 스타일에 해당하는 loss와 콘텐츠에 대한 loss를 각각의 가중치를 곱해서 더한 값을 활용한다. 스타일 loss는 모든 layer에서 발생하는데, 이는 layer를 거듭할 수록 receptive field가 달라지기 때문이다. 즉 세밀한 영역부터 넓은 영역까지의 스타일을 학습하기 위함이다. 반면 콘텐츠 loss는 특정 layer에서만 계산되는데, 이는 콘텐츠의 형태를 잘 보존하면서, 스타일을 잘 입힐 수 있는 위치를 실험적으로 찾은 것으로 보인다. 다음으로 Optimizer는 기존 Adam, SGD와 다른 L-BFGS 를 사용한다. 이는 2차 미분을 활용한 경사하강법을 계산속도를 올리기 위해, 근사한 Optimizer이다.